In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

def classification_training(data):
    
    # Подготовка целевой переменной
    y = (data['mental_wellness_index_0_100'] >= 15).astype(int)

    # Подготовка признаков - все, кроме целевого ('mental_wellness_index_0_100') и идентификатора ('user_id')
    feature_columns = [col for col in data.columns if col not in ['mental_wellness_index_0_100', 'user_id']]
    X = data[feature_columns].copy()

    # Обработка категориальных признаков с использованием LabelEncoder
    categorical_features = ['gender', 'occupation', 'work_mode']
    le_dict = {}
    for feature in categorical_features:
        if feature in X.columns:
            le = LabelEncoder()
            X.loc[:, feature] = le.fit_transform(X[feature].astype(str))
            le_dict[feature] = le

    # Разделение датасета на тренировочную и тестовую выборку
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.25,
        shuffle=True,
        random_state=37
    )

    # Создание словаря-сетки для GridSearchCV
    param_grid = {
        'n_estimators': [50, 100, 200], 
        'max_depth': [None, 10, 20], 
        'min_samples_split': [2, 5, 10], 
        'min_samples_leaf': [1, 2, 4], 
        'max_features': ['sqrt', 'log2'] 
    }

    # Создание экземпляра класса случайного леса
    rf_base = RandomForestClassifier(
        random_state=37, # Для воспроизводимости
        n_jobs=-1 # Использовать все доступные ядра
    )

    # Выполнение поиска оптимального набора гиперпараметров
    grid_search = GridSearchCV(
        estimator=rf_base,
        param_grid=param_grid,
        scoring='f1', 
        cv=5, 
        n_jobs=-1, 
        verbose=0 
    )

    # Обучаем GridSearchCV на тренировочных данных
    grid_search.fit(X_train, y_train)

    # Вывод оптимального набора гиперпараметров
    print("Л:")
    for param, value in grid_search.best_params_.items():
        print(f"  {param}: {value}")

    # Обучение финальной модели с оптимальным набором гиперпараметров
    best_rf_model = grid_search.best_estimator_

    # Предсказания и оценка финальной модели на тестовой выборке
    y_pred = best_rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Вывод значений метрик
    print(f"RF: {accuracy:.4f}; {f1:.4f}")

data = pd.read_csv("DB_3_cleaned.csv") # Замените на путь к вашему датасету
classification_training(data)


Л:
  max_depth: None
  max_features: sqrt
  min_samples_leaf: 2
  min_samples_split: 10
  n_estimators: 50
RF: 0.9596; 0.9545
